<a href="https://colab.research.google.com/github/mbagci06/lung_covid/blob/main/test_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# BASICS 
import os
import numpy as np
import torch

# MODEL and TRAIN
import torchvision
import torchvision.models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import random
from skimage import io
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import pyplot as plt
from torch import optim
from PIL import Image

In [2]:
! git clone https://github.com/mbagci06/lung_covid
! pip install kaggle
! mkdir ~/.kaggle
! cp lung_covid/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d anasmohammedtahir/covidqu

Cloning into 'lung_covid'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 142 (delta 18), reused 0 (delta 0), pack-reused 100
Receiving objects: 100% (142/142), 1.18 MiB | 7.74 MiB/s, done.
Resolving deltas: 100% (66/66), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
100% 1.15G/1.15G [00:11<00:00, 105MB/s] 
100% 1.15G/1.15G [00:11<00:00, 109MB/s]


In [3]:
!unzip "/content/covidqu.zip" -d "/content/archive/" &> /dev/null

In [4]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!gdown --folder https://drive.google.com/drive/folders/1MIzhxSou4TRtQTcG3AADqLag28nj3FrV?usp=sharing

Retrieving folder list
Processing file 1tcMX2dtdHO17cdE09bmRZDTg0zovvtUH deeplab_lung_30_epo.pth
Processing file 1RM9kH2BAG7yq-L2larWPS23levRhgMRz deeplab_lung_30_epo.txt
Processing file 18cQG7yKb3q-AVsJB8gszLKePB_x5PqnT deeplab_transfer_covid_60_epo.pth
Processing file 1KWM07yB07IoKfoXBOZr7Y9xqmhX5WnqM deeplab_transfer_covid_60_epo.txt
Processing file 1NdJZO_B3uoQ7pmEVS3ClGv0dL3FoTlS3 UNet_covid_50_epo.txt
Processing file 1XBugwkZDCu85Nkd6P85_1Wtca98W7-hk UNet_covid_50_epoch.pt
Processing file 1I8X5OfFqL6hhGMKcdm0FMSmJyn9ongr8 UNet_model_lung_pre_30_epo.txt
Processing file 1mVDHpSxnoMnDn1o34gbQ5ISKmUgVX67O UNet_model_lung_pre_30_epoch.pt
Processing file 1MYWXCbnlKgWOVzYH3mjCBWM0L2DAzNp9 UNet_transfer_covid_90_epo.txt
Processing file 1-6sShM8RjJ1nqiU3dGPTZ3E904yqu1xa UNet_transfer_covid_90_epoch.pt
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1tcMX2dtdHO17cdE09bmRZDTg0zovvtUH
To:

In [6]:
%run lung_covid/functions.ipynb
%run lung_covid/DeepLab.ipynb
%run lung_covid/unet.ipynb

In [7]:
# The device looks for GPU if cannot find goes with CPU
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [8]:

test_lung=Lung_Dataset_RGB(split = "Test",class_folder='lung',sample_size=25)
tra_lung_size=test_lung.__len__()
print("Number of the Lung test images are ",tra_lung_size)
test_covid=Lung_Dataset_RGB(split = "Test",class_folder='covid',sample_size=25)
tra_covid_size=test_covid.__len__()
print("Number of the Covid test images are ",tra_covid_size)

Number of the Lung test images are  6788
Number of the Covid test images are  1166


In [9]:
batch_size=12
# loading images and masks for mini batches
test_lung_loader = torch.utils.data.DataLoader(
        test_lung, batch_size=batch_size,  num_workers=2, drop_last=True)

test_covid_loader = torch.utils.data.DataLoader(
        test_covid, batch_size=batch_size,  num_workers=2, drop_last=True)

- UNet_model_lung_pre_30_epoch.pt
- UNet_transfer_covid_90_epoch.pt
- deeplab_lung_30_epo.pth

In [ ]:
# Pretraned model Lung U-Net 
net_l = Unet_Res().to(device)
checkpoint = torch.load('ECE228_models_texts/UNet_model_lung_pre_30_epoch.pt',map_location=torch.device(device=device))
net_l.load_state_dict(checkpoint['model_state_dict'])
net_l.eval()

In [11]:
# # Pretraned model covid U-Net 
# net_c = Unet_Res().to(device)
# checkpoint = torch.load('ECE228_models_texts/'+file_name')
# net_c.load_state_dict(checkpoint['model_state_dict'])

In [12]:
# # Trabsfer learning Lung u-Net 
# net_tl = Unet_Res().to(device)
# checkpoint = torch.load('ECE228_models_texts/'+file_name')
# net_tl.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
# Trabsfer learning Covid U-Net 
net_tc = Unet_Res().to(device)
checkpoint = torch.load('ECE228_models_texts/UNet_transfer_covid_90_epoch.pt',map_location=torch.device(device=device))
net_tc.load_state_dict(checkpoint['model_state_dict'])
net_tc.eval()

In [ ]:
# Pretraned model Lung Deeplab
deep_l = DeepLabv3_plus(nInputChannels=3, n_classes=2, os=8, pretrained=True).to(device=device)
checkpoint = torch.load('ECE228_models_texts/deeplab_lung_30_epo.pth',map_location=torch.device(device=device))
deep_l.load_state_dict(checkpoint['model_state_dict'])
deep_l.eval()

In [20]:
!nvidia-smi

Wed Jun  8 09:07:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    43W / 250W |   4457MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
acc_bin=0
acc_dice=0
acc_iou=0
for batch_idx, sample in enumerate(test_lung_loader):
    # Validation data preparing  and calling 

    # Call iamge and mask
    imgs , true_masks = sample['image'],sample['mask']

    imgs = imgs.to(device=device)
    true_masks=Negative_mask(true_masks,imgs.size(dim=0))
    true_masks = true_masks.to(device=device)
    # zero the parameter gradients

    with torch.set_grad_enabled(False):
        # Validation evaluation 
        masks_pred = net_l(imgs)

        acc_dice += dice( masks_pred, true_masks)
        acc_bin +=binary_acc( masks_pred, true_masks).item()
        acc_iou += iou( masks_pred, true_masks)


nomi=(tra_lung_size/batch_size)
print('Lung Acc Bin:',(acc_bin)/(tra_lung_size/batch_size))
print('Lung Acc Dice :',(acc_dice)/(tra_lung_size/batch_size))
print('Lung Acc iou :',(acc_iou)/(tra_lung_size/batch_size))

Lung Acc Bin: 0.9855007819730391
Lung Acc Dice : 0.9854472773034361
Lung Acc iou : tensor(0.9725, device='cuda:0')


In [19]:
acc_bin=0
acc_dice=0
acc_iou=0
for batch_idx, sample in enumerate(test_lung_loader):
    # Validation data preparing  and calling 

    # Call iamge and mask
    imgs , true_masks = sample['image'],sample['mask']

    imgs = imgs.to(device=device)
    true_masks=Negative_mask(true_masks,imgs.size(dim=0))
    true_masks = true_masks.to(device=device)
    # zero the parameter gradients

    with torch.set_grad_enabled(False):
        # Validation evaluation 
        masks_pred = deep_l(imgs)

        acc_dice += dice( masks_pred, true_masks)
        acc_bin +=binary_acc( masks_pred, true_masks).item()
        acc_iou += iou( masks_pred, true_masks)


nomi=(tra_lung_size/batch_size)
print('Lung Acc Bin:',(acc_bin)/(tra_lung_size/batch_size))
print('Lung Acc Dice :',(acc_dice)/(tra_lung_size/batch_size))
print('Lung Acc iou :',(acc_iou)/(tra_lung_size/batch_size))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3704: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Lung Acc Bin: 0.9851749868834097
Lung Acc Dice : 0.9851334514727506
Lung Acc iou : tensor(0.9719, device='cuda:0')


In [18]:
acc_bin=0
acc_dice=0
acc_iou=0
for batch_idx, sample in enumerate(test_covid_loader):
    # Validation data preparing  and calling 

    # Call iamge and mask
    imgs , true_masks = sample['image'],sample['mask']

    imgs = imgs.to(device=device)
    true_masks=Negative_mask(true_masks,imgs.size(dim=0))
    true_masks = true_masks.to(device=device)
    # zero the parameter gradients

    with torch.set_grad_enabled(False):
        # Validation evaluation 
        masks_pred = net_tc(imgs)

        acc_dice += dice( masks_pred, true_masks)
        acc_bin +=binary_acc( masks_pred, true_masks).item()
        acc_iou += iou( masks_pred, true_masks)


nomi=(tra_covid_size/batch_size)
print('covid transfer Acc Bin:',(acc_bin)/(tra_covid_size/batch_size))
print('covid transfer Dice :',(acc_dice)/(tra_covid_size/batch_size))
print('covid transfer iou :',(acc_iou)/(tra_covid_size/batch_size))

covid transfer Acc Bin: 0.9733729847095024
covid transfer Dice : 0.9732564961726219
covid transfer iou : tensor(0.9506, device='cuda:0')
